<a href="https://colab.research.google.com/github/DivinSphere/Desempenho-de-Modelos-de-IA-com-Python/blob/main/Avalia%C3%A7%C3%A3o_de_Desempenho_de_Modelos_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**--------Implementação rápida de avaliação de Desempenho de diversos modelos de Classificação----------**

Todos os modelos listados são treinados e testados com o conjunto de treinamento disponível em fetch_openml("adult")

O dataframe final é ordenado pela Acurácia e então, pode-se observar os melhores modelos para os piores modelos nessas condições atuais e no conjunto de dados utilizados.


**--------Informações Desenvolvedor----------**

Código por Ismael Weslley Neves de Brito

Linkedin: https://www.linkedin.com/in/ismael-weslley-a915b2236/

Github: https://github.com/DivinSphere


**--------Explicação da base de dados----------**

A base de dados "adult" no OpenML é um conjunto de dados que contém informações sobre indivíduos adultos nos Estados Unidos, incluindo informações demográficas, educacionais e de renda. 

Ele é frequentemente usado como um exemplo de conjunto de dados para aprendizado supervisionado de classificação. 

A finalidade principal da base de dados é ser usado como um dataset para o treinamento e teste de algoritmos de machine learning.

A variável de resultado é 'renda anual': '>50K' (favorável) ou '<=50K' (desfavorável).

Dessa base de dados, 40 mil registros foram utilizados para o treinamento, para execuções mais rápidas, experimente diminuir o valor para 20 mil, 10 mil ou 5 mil.

**--------Sobre os modelos utilizados----------**

Os modelos foram escolhidos sem nenhum tipo de critério, alguns escolhidos apenas por serem muito conhecidos, outros nem tanto. Não houve algum critério de "Esse modelo vai ser melhor para essa situação" ou coisa do tipo, modelos aleatórios foram escolhidos para esse experimento.

É importante notar que o desempenho de um modelo pode variar dependendo do conjunto de dados utilizado e das configurações específicas do modelo. Embora um modelo possa se sair melhor em um experimento específico, isso não significa necessariamente que ele será sempre superior a outros modelos.

Além disso, os modelos utilizados neste código foram instanciados com suas configurações originais, sem a adição ou modificação de parâmetros para melhorar o desempenho. Isso é para garantir que os resultados sejam comparáveis entre os modelos e que as descobertas sejam baseadas nas configurações padrão dos algoritmos.

Os valores e resultados dos modelos podem variar a cada vez que o código é executado, fazendo com que em determinado momento um algoritmo seja mais eficiente que outro, portanto é importante sempre verificar múltiplas vezes para ter certeza da eficácia do modelo.

In [1]:
# Importação de todas as bibliotecas necessárias para a execução correta do código
# Adiciona todos os modelos de classificadores necessários

import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, roc_auc_score, roc_curve)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import (LogisticRegression, Perceptron, RidgeClassifier, PassiveAggressiveClassifier, SGDClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_openml
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB

/usr/local/lib/python3.8/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [2]:
# Cria a função que vai retornar as estatisticas de cada modelo após executado
def calculate_attributes(y_true, y_pred):
    return {
        # a acurácia é uma medida de quão precisos são os resultados do modelo, calculada dividindo o número de previsões corretas pelo número total de previsões.
        "acurácia": accuracy_score(y_true, y_pred),
        # O F1-score é uma média harmônica entre precisão e recall. Ele é útil quando a distribuição de classe é desbalanceada.
        "f1-score": f1_score(y_true, y_pred, average='macro'),
        # o recall é uma medida de quão completamente o modelo identifica as instâncias de uma classe específica, calculada dividindo o número de instâncias da classe específica identificadas corretamente pelo número total de instâncias da classe específica.
        "recall": recall_score(y_true, y_pred, average='macro', zero_division=1),
        # precisão é uma medida de quão frequentemente o modelo faz uma previsão correta para uma classe específica, calculada dividindo o número de previsões corretas para uma classe específica pelo número total de previsões para essa classe.
        "precisão": precision_score(y_true, y_pred, average='macro', zero_division=1),
        # A matriz de confusão é uma tabela que mostra as previsões corretas e incorretas do modelo para cada classe. Ele é útil para entender como o modelo está se comportando em relação a cada classe
        "matriz de confusão": confusion_matrix(y_true, y_pred),
        # ROC AUC é uma medida da capacidade do modelo de discriminar entre classes. Ele é calculado com base na curva ROC (receiver operating characteristic).
        "roc_auc": roc_auc_score(y_true, y_pred),
        # A curva ROC é um gráfico que mostra a relação entre a taxa de verdadeiros positivos e a taxa de falsos positivos para diferentes limiares de classificação. Ela é útil para entender como o modelo se comporta em relação a diferentes níveis de sensibilidade e especificidade.
        "curva roc": roc_curve(y_true, y_pred)
    }

In [3]:
# Seleção de todos os modelos que serão usados para se fazer a comparação das execuções e das estatisticas
models = [
    # Regressão Logística: é um algoritmo de classificação linear que é usado para prever a probabilidade de uma variável categórica dependente.
    ("Regressão Logística", LogisticRegression()),
    # Árvore de Decisão: é um algoritmo de aprendizado supervisionado que é usado para classificação e regressão. Ele funciona criando uma árvore de decisão com nós de decisão e folhas de classificação.
    ("Árvore de Decisão", DecisionTreeClassifier()),
    # K Vizinhos mais Próximos: é um algoritmo de aprendizado supervisionado baseado em instância que classifica novos exemplos baseando-se na maioria das classificações dos k vizinhos mais próximos.
    ("K Vizinhos mais Próximos", KNeighborsClassifier()),
    # Floresta Aleatória: é um conjunto de árvores de decisão, onde cada árvore é construída a partir de um subconjunto aleatório de exemplos e recursos do conjunto de dados.
    ("Floresta Aleatória", RandomForestClassifier()),
    # Gradient Boosting: é um algoritmo de aprendizado supervisionado que usa uma série de árvores de decisão para melhorar a precisão do modelo.
    ("Gradient Boosting", GradientBoostingClassifier()),
    # SVC: é uma classe de algoritmos de aprendizado de máquina supervisionado para classificação e regressão, baseado em algoritmos de classificação binária.
    ("SVC", SVC()),
    # Naive Bayes: é um conjunto de algoritmos de aprendizado supervisionado baseados na aplicação do teorema de Bayes com a suposição de independência entre os recursos.
    ("Naive Bayes", GaussianNB()),
    # Perceptron: é um algoritmo de aprendizado de máquina supervisionado usado para classificação binária.
    ("Perceptron",Perceptron()),
    # Rede Neural Multicamada: é um algoritmo de aprendizado de máquina supervisionado que consiste em várias camadas de nós de processamento.
    ("Rede Neural Multicamada", MLPClassifier()),
    # AdaBoost: é um algoritmo de aprendizado supervisionado que combina vários classificadores fracos para criar um classificador forte.
    ("AdaBoost", AdaBoostClassifier()),
    # Bagging: é um algoritmo de aprendizado supervisionado que consiste em aplicar o mesmo classificador várias vezes com subconjuntos aleatórios do conjunto de dados.
    ("Bagging", BaggingClassifier()),
    # ExtraTreesClassifier: é uma variação do algoritmo Random Forest, onde cada árvore é construída a partir de um subconjunto aleatório de exemplos e recursos do conjunto de dados e todos os recursos são considerados em cada divisão.
    ("ExtraTreesClassifier", ExtraTreesClassifier()),
    # XGBoost: é uma biblioteca de aprendizado de máquina de código aberto que implementa o algoritmo de gradient boosting de árvore de decisão.
    ("XGBoost", XGBClassifier()),
    # LightGBM: é uma biblioteca de aprendizado de máquina de código aberto que implementa o algoritmo de gradient boosting de árvore de decisão otimizado para grandes conjuntos de dados.
    ("LightGBM", LGBMClassifier()),
    # Ridge: é um algoritmo de classificação linear que é usado para prever a probabilidade de uma variável categórica dependente, ele inclui uma regularização L2.
    ("Ridge", RidgeClassifier()),
    # PassiveAggressiveClassifier: é um algoritmo de aprendizado supervisionado baseado em instância que atualiza o modelo de acordo com os erros cometidos.
    ("PassiveAggressiveClassifier", PassiveAggressiveClassifier()),
    # SGDClassifier: é uma classe de algoritmos de aprendizado de máquina supervisionado para classificação e regressão, baseado em algoritmos de descida de gradiente estocástico.
    ("SGDClassifier",SGDClassifier()),
    # BernoulliNB: é um algoritmo de classificação baseado em Bayes, específico para dados binários. Ele assume que todas as características são binárias e aplica o teorema de Bayes com a suposição de independência entre os recursos.
    ("BernoulliNB",BernoulliNB()),
    # HistGradientBoostingClassifier: é uma variação do algoritmo Gradient Boosting que usa técnicas de histograma para otimizar a construção das árvores. 
    ("HistGradientBoostingClassifier",HistGradientBoostingClassifier()),
    # DummyClassifier: é um classificador "dummy" que permite definir uma estratégia para atribuir classes aos exemplos. Ele pode ser usado como uma base de comparação para outros classificadores.
    ("DummyClassifier",DummyClassifier()),
    # MultinomialNB: é uma variação do algoritmo Naive Bayes, específico para dados contínuos. Ele assume que todas as características são contínuas e aplica o teorema de Bayes com a suposição de independência entre os recursos.
    ("MultinomialNB",MultinomialNB())
]

In [4]:
# carregando o dataset com uma amostra de valores
data = fetch_openml("adult",version=1)
df = pd.DataFrame(data.data, columns=data.feature_names)
df = df.sample(n=40000)
data.target = data.target[df.index]

# tratando as variaveis categóricas
df = pd.get_dummies(df, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

# separando em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(df, data.target, test_size=0.2, random_state=42)

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

In [5]:
# Treina cada modelo com os dados
# Retorna suas estatisticas para cada execução

results = {}

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = calculate_attributes(y_test, y_pred)

# Transforma em um DataFrame Pandas
# Retorna com os modelos organizados por ordem de Acurácia
df_modelos = pd.DataFrame.from_dict(results).T
df_modelos = df_modelos.sort_values("acurácia", ascending=False)
df_modelos

,acurácia,f1-score,recall,precisão,matriz de confusão,roc_auc,curva roc
LightGBM,0.855875,0.791845,0.776329,0.81312,"[[5642, 421], [732, 1205]]",0.776329,"([0.0, 0.0694375721589972, 1.0], [0.0, 0.62209..."
HistGradientBoostingClassifier,0.85575,0.791539,0.775895,0.813059,"[[5643, 420], [734, 1203]]",0.775895,"([0.0, 0.06927263730826323, 1.0], [0.0, 0.6210..."
Gradient Boosting,0.85375,0.78526,0.76632,0.813614,"[[5674, 389], [781, 1156]]",0.76632,"([0.0, 0.06415965693551047, 1.0], [0.0, 0.5967..."
XGBoost,0.853,0.783168,0.763366,0.813547,"[[5682, 381], [795, 1142]]",0.763366,"([0.0, 0.0628401781296388, 1.0], [0.0, 0.58957..."
AdaBoost,0.851125,0.784118,0.768101,0.806618,"[[5633, 430], [761, 1176]]",0.768101,"([0.0, 0.07092198581560284, 1.0], [0.0, 0.6071..."
Ridge,0.844625,0.761496,0.735882,0.810409,"[[5740, 323], [920, 1017]]",0.735882,"([0.0, 0.05327395678706911, 1.0], [0.0, 0.5250..."
Floresta Aleatória,0.828125,0.758614,0.751171,0.767501,"[[5459, 604], [771, 1166]]",0.751171,"([0.0, 0.0996206498433119, 1.0], [0.0, 0.60196..."
Bagging,0.826375,0.750038,0.737544,0.767219,"[[5516, 547], [842, 1095]]",0.737544,"([0.0, 0.09021936335147616, 1.0], [0.0, 0.5653..."
ExtraTreesClassifier,0.816125,0.745002,0.740794,0.749687,"[[5377, 686], [785, 1152]]",0.740794,"([0.0, 0.11314530760349661, 1.0], [0.0, 0.5947..."
Árvore de Decisão,0.7965,0.725247,0.72732,0.723292,"[[5223, 840], [788, 1149]]",0.72732,"([0.0, 0.13854527461652646, 1.0], [0.0, 0.5931..."
